In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(
    "../data/consumption.csv", usecols=["prediction_unit_id", "datetime", "target"]
)[["prediction_unit_id", "datetime", "target"]].rename(
    columns={"prediction_unit_id": "unique_id", "datetime": "ds", "target": "y"}
)
df["ds"] = pd.to_datetime(df["ds"])
df.head()

,unique_id,ds,y
0,0,2021-09-01,96.590
1,1,2021-09-01,17.314
2,2,2021-09-01,656.859
3,3,2021-09-01,59.000
4,4,2021-09-01,501.760


In [3]:
df["y"] = df["y"].interpolate(method="linear")
df.isna().sum()

unique_id    0
ds           0
y            0
dtype: int64

# Train/Test split

In [4]:
# taking the last 48 hours for test
for i in df["unique_id"].unique():
    if i == df["unique_id"].unique()[0]:
        df_test = df[df["unique_id"]==i][-48:]
        continue
    df_test = pd.concat([df_test, df[df["unique_id"]==i][-48:]])
    
df_test.sort_index(inplace=True)
print(df_test.shape)
df_test.head()

(3312, 3)


,unique_id,ds,y
906150,26,2023-03-28 00:00:00,5.137
906217,26,2023-03-28 01:00:00,4.212
906284,26,2023-03-28 02:00:00,4.917
906351,26,2023-03-28 03:00:00,4.195
906418,26,2023-03-28 04:00:00,4.518


In [5]:
train_idx = [idx for idx in df.index if idx not in df_test.index]
df_train = df.loc[train_idx]
df_train.shape
df_train.head()

,unique_id,ds,y
0,0,2021-09-01,96.590
1,1,2021-09-01,17.314
2,2,2021-09-01,656.859
3,3,2021-09-01,59.000
4,4,2021-09-01,501.760


In [6]:
df.shape[0] == df_train.shape[0] + df_test.shape[0]

True

In [7]:
df.shape[1] == df_train.shape[1] == df_test.shape[1]

True

# Set features and target

In [8]:
def get_features(df):
    y = pd.concat(
        [
            df[["unique_id", "ds"]],
            df.groupby('unique_id')["y"].shift(-1).rename("lead1"),
            df.groupby('unique_id')["y"].shift(-2).rename("lead2"),
        ], axis=1
    ).dropna()
    X = pd.concat(
        [
            df,
            df.groupby('unique_id')["y"].shift(1).rename("lag1"),
            df.groupby('unique_id')["y"].shift(2).rename("lag2"),
        ], axis=1
    ).dropna()
    y, X = y.align(X, axis=0, join="inner")
    return X, y

In [9]:
X_train, y_train = get_features(df_train)
display("X_train", X_train.head())
display("y_train", y_train.head())

'X_train'

,unique_id,ds,y,lag1,lag2
122,0,2021-09-01 02:00:00,91.594,77.691,96.590
123,1,2021-09-01 02:00:00,16.510,15.872,17.314
124,2,2021-09-01 02:00:00,598.450,595.498,656.859
125,3,2021-09-01 02:00:00,63.100,61.600,59.000
126,4,2021-09-01 02:00:00,458.562,486.297,501.760


'y_train'

,unique_id,ds,lead1,lead2
122,0,2021-09-01 02:00:00,87.955,88.184
123,1,2021-09-01 02:00:00,14.271,18.225
124,2,2021-09-01 02:00:00,622.824,607.308
125,3,2021-09-01 02:00:00,64.000,60.500
126,4,2021-09-01 02:00:00,511.794,520.318


We verify that features have been correctly created :

In [13]:
df_merge = pd.merge(X_train, y_train, on=["unique_id", "ds"])[["unique_id", "ds", "lead2", "lead1", "y", "lag1", "lag2"]]
df_merge[df_merge["unique_id"]==0]

,unique_id,ds,lead2,lead1,y,lag1,lag2
0,0,2021-09-01 02:00:00,88.184,87.955,91.594,77.691,96.590
61,0,2021-09-01 03:00:00,89.781,88.184,87.955,91.594,77.691
122,0,2021-09-01 04:00:00,96.481,89.781,88.184,87.955,91.594
183,0,2021-09-01 05:00:00,94.592,96.481,89.781,88.184,87.955
244,0,2021-09-01 06:00:00,77.308,94.592,96.481,89.781,88.184
...,...,...,...,...,...,...,...
1005263,0,2023-05-29 17:00:00,367.135,209.099,148.756,108.792,80.552
1005328,0,2023-05-29 18:00:00,469.458,367.135,209.099,148.756,108.792
1005393,0,2023-05-29 19:00:00,580.554,469.458,367.135,209.099,148.756
1005458,0,2023-05-29 20:00:00,624.870,580.554,469.458,367.135,209.099


Note : ``ds`` is the date and time of the last measured value ``y``

In [14]:
X_test, y_test = get_features(df_test)
display("X_test", X_test.head())
display("y_test", y_test.head())

'X_test'

,unique_id,ds,y,lag1,lag2
906284,26,2023-03-28 02:00:00,4.917,4.212,5.137
906351,26,2023-03-28 03:00:00,4.195,4.917,4.212
906418,26,2023-03-28 04:00:00,4.518,4.195,4.917
906485,26,2023-03-28 05:00:00,5.800,4.518,4.195
906552,26,2023-03-28 06:00:00,5.966,5.800,4.518


'y_test'

,unique_id,ds,lead1,lead2
906284,26,2023-03-28 02:00:00,4.195,4.518
906351,26,2023-03-28 03:00:00,4.518,5.800
906418,26,2023-03-28 04:00:00,5.800,5.966
906485,26,2023-03-28 05:00:00,5.966,5.427
906552,26,2023-03-28 06:00:00,5.427,4.221


# Training

In [17]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler

In [18]:
# to drop 'unique_id' and 'ds' columns from X when using fit and predict
class columnDropperTransformer():
    def __init__(self, columns):
        self.columns = columns

    def transform(self, X, y=None):
        return X.drop(self.columns, axis=1)

    def fit(self, X, y=None):
        return self

In [19]:
pipeline = Pipeline(
    [
        ("columnDropper", columnDropperTransformer(columns=['unique_id','ds'])),
        ("scaler", MaxAbsScaler()),
        ("lgb", MultiOutputRegressor(LGBMRegressor())),
    ]
)

In [21]:
pipeline.fit(X_train, y_train.drop(columns=["unique_id", "ds"]))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009193 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 1005588, number of used features: 3
[LightGBM] [Info] Start training from score 460.988094
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 1005588, number of used features: 3
[LightGBM] [Info] Start training from score 460.997483


Pipeline(steps=[('columnDropper',
                 <__main__.columnDropperTransformer object at 0x000002D4365EAE90>),
                ('scaler', MaxAbsScaler()),
                ('lgb', MultiOutputRegressor(estimator=LGBMRegressor()))])

# Inference

In [51]:
input_data = X_test.groupby("unique_id").head(1)
predictions = pipeline.predict(input_data)
preds_df = pd.DataFrame(predictions, index=input_data.index, columns=["lead1", "lead2"])
preds_df

,lead1,lead2
906284,6.336469,7.797782
980628,198.389771,200.179522
995152,5.960211,7.312226
999946,3.799106,5.175203
1006186,435.094947,429.744905
...,...,...
1006246,33.566850,35.215187
1006247,76.763052,79.558248
1006248,172.384871,174.256769
1006249,33.989368,35.383654


In [52]:
pd.concat(
    [input_data.iloc[:, :2], preds_df],
    axis=1
)

,unique_id,ds,lead1,lead2
906284,26,2023-03-28 02:00:00,6.336469,7.797782
980628,41,2023-05-14 02:00:00,198.389771,200.179522
995152,47,2023-05-23 02:00:00,5.960211,7.312226
999946,44,2023-05-26 02:00:00,3.799106,5.175203
1006186,0,2023-05-30 02:00:00,435.094947,429.744905
...,...,...,...,...
1006246,57,2023-05-30 02:00:00,33.566850,35.215187
1006247,58,2023-05-30 02:00:00,76.763052,79.558248
1006248,64,2023-05-30 02:00:00,172.384871,174.256769
1006249,59,2023-05-30 02:00:00,33.989368,35.383654


In [89]:
row = input_data[:1]
row[["y", "lag1", "lag2"]].values.reshape(-1,).tolist()

[4.917, 4.212, 5.137]

In [102]:
def combine_columns(row):
    return list((row['y'], row['lag1'], row["lag2"]))
    # return row[["y", "lag1", "lag2"]].values.reshape(-1,).tolist()

# Apply the custom function to create a new column 'Combined'
input_data["values"] = input_data.apply(combine_columns, axis=1)
input_data = input_data.drop(columns=["y", "lag1", "lag2"])
input_data

C:\Users\gabriel.chehade\AppData\Local\Temp\ipykernel_13104\599344338.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["values"] = input_data.apply(combine_columns, axis=1)


,unique_id,ds,values
906284,26,2023-03-28 02:00:00,"[4.917, 4.212, 5.137]"
980628,41,2023-05-14 02:00:00,"[195.46, 220.102, 190.319]"
995152,47,2023-05-23 02:00:00,"[4.807, 1.891, 1.894]"
999946,44,2023-05-26 02:00:00,"[2.152, 6.548, 2.362]"
1006186,0,2023-05-30 02:00:00,"[440.428, 465.981, 461.672]"
...,...,...,...
1006246,57,2023-05-30 02:00:00,"[32.837, 30.152, 36.91]"
1006247,58,2023-05-30 02:00:00,"[79.38, 79.613, 95.835]"
1006248,64,2023-05-30 02:00:00,"[175.893, 187.635, 181.786]"
1006249,59,2023-05-30 02:00:00,"[32.823, 27.825, 28.832]"


In [ ]:
{'split', 'records', 'index', 'columns',
          'values', 'table'}

In [110]:
data_json = """
[
    {
        "unique_id":1,
        "ds":1630454400000,
        "values":[17.314, 96.59],
    },
    {
        "unique_id":2,
        "ds":1630454400000,
        "values":[656.859, null],
    }
]
"""

In [148]:
data_json = input_data.to_json(orient="records", indent=4)
print(data_json[:460])

[
    {
        "unique_id":26,
        "ds":1679968800000,
        "values":[
            4.917,
            4.212,
            5.137
        ]
    },
    {
        "unique_id":41,
        "ds":1684029600000,
        "values":[
            195.46,
            220.102,
            190.319
        ]
    },
    {
        "unique_id":47,
        "ds":1684807200000,
        "values":[
            4.807,
            1.891,
            1.894
        ]
    },
   


In [115]:
from io import StringIO

input_df = pd.read_json(StringIO(data_json))
input_df

,unique_id,ds,values
0,26,1679968800000,"[4.917, 4.212, 5.1370000000000005]"
1,41,1684029600000,"[195.46, 220.102, 190.319]"
2,47,1684807200000,"[4.807, 1.891, 1.8940000000000001]"
3,44,1685066400000,"[2.152, 6.548, 2.362]"
4,0,1685412000000,"[440.428, 465.981, 461.672]"
...,...,...,...
64,57,1685412000000,"[32.837, 30.152, 36.91]"
65,58,1685412000000,"[79.38, 79.613, 95.835]"
66,64,1685412000000,"[175.893, 187.635, 181.786]"
67,59,1685412000000,"[32.823, 27.825, 28.832]"


In [126]:
input_df[["y", "lag1", "lag2"]] = pd.DataFrame(input_df["values"].tolist(), index= input_df.index)
input_df = input_df.drop(columns="values")
input_df

,unique_id,ds,y,lag1,lag2
0,26,1679968800000,4.917,4.212,5.137
1,41,1684029600000,195.460,220.102,190.319
2,47,1684807200000,4.807,1.891,1.894
3,44,1685066400000,2.152,6.548,2.362
4,0,1685412000000,440.428,465.981,461.672
...,...,...,...,...,...
64,57,1685412000000,32.837,30.152,36.910
65,58,1685412000000,79.380,79.613,95.835
66,64,1685412000000,175.893,187.635,181.786
67,59,1685412000000,32.823,27.825,28.832


In [128]:
# input_df = pd.read_json(StringIO(data_json)).explode("values")#.fillna(value=0)#.reset_index()
input_df["ds"] = pd.to_datetime(input_df["ds"], unit="ms")
input_df

,unique_id,ds,y,lag1,lag2
0,26,2023-03-28 02:00:00,4.917,4.212,5.137
1,41,2023-05-14 02:00:00,195.460,220.102,190.319
2,47,2023-05-23 02:00:00,4.807,1.891,1.894
3,44,2023-05-26 02:00:00,2.152,6.548,2.362
4,0,2023-05-30 02:00:00,440.428,465.981,461.672
...,...,...,...,...,...
64,57,2023-05-30 02:00:00,32.837,30.152,36.910
65,58,2023-05-30 02:00:00,79.380,79.613,95.835
66,64,2023-05-30 02:00:00,175.893,187.635,181.786
67,59,2023-05-30 02:00:00,32.823,27.825,28.832


In [88]:
df_input.fillna(value=np.nan)

/tmp/ipykernel_1994/390412465.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_input.fillna(value=np.nan)


,unique_id,ds,values
0,1,2021-09-01,17.314
0,1,2021-09-01,96.590
1,2,2021-09-01,656.859
1,2,2021-09-01,NaN


In [89]:
#pd.DataFrame(pipeline.predict(X_test.head(10)), columns=y_test.columns)
pipeline.predict(df_input)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- values
Feature names seen at fit time, yet now missing:
- lag_0
- lag_1


# Visualization

# Save model

In [213]:
import joblib

joblib.dump(model, 'model_test.joblib')

['model_test.joblib']

Load the model for test

In [214]:
with open('model_test.joblib', 'rb') as file:
    loaded_model = joblib.load(file)
    
loaded_model.predict(X_test.iloc[:10, 2:]) == model.predict(X_test.iloc[:10, 2:])

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])

# App deployment

In [63]:
from flask import jsonify

In [ ]:
import joblib
import json
from flask import Flask, request
import numpy as np
import pandas as pd

model = None

def load_model():
    global model
    # model variable refers to the global variable
    with open('model_test.joblib', 'rb') as file:
        model = joblib.load(file)
        
        
app = Flask(__name__)

